In [1]:
import torch

# Check if CUDA is available
if torch.cuda.is_available():
    print("CUDA is available.")
else:
    print("CUDA is not available. Check if your GPU drivers are properly installed.")


CUDA is available.


In [2]:
# !pip install datasets>=2.6.1
# !pip install transformers
# !pip install librosa
# !pip install jiwer
# !pip install evaluate>=0.30
# #!pip install gradio
# !pip install -U accelerate

#from 20.5.2024
# !pip install audiomentations[extras]
# !pip install mutagen
# !pip install srt

In [3]:
import os
os.chdir('/app/')

In [4]:
from huggingface_hub import login
# load the token from txt file
with open("./tokens/HF_token.txt", "r") as f:
    HF_TOKEN = f.read().strip() # strip() removes the trailing "\n" if it exists
login(token=HF_TOKEN)


The token has not been saved to the git credentials helper. Pass `add_to_git_credential=True` in this function directly or `--add-to-git-credential` if using via `huggingface-cli` if you want to set the git credential as well.


/opt/conda/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Token is valid (permission: write).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [5]:
import librosa
import random
import numpy as np
import IPython.display as ipd
import pickle
import pandas as pd
from datasets import Dataset
from datasets import Audio
from torch.utils.data import ConcatDataset
from transformers import WhisperProcessor
import mutagen.mp3
from tqdm import tqdm
import json
from audiomentations import Compose, AddGaussianNoise, TimeStretch, PitchShift, Shift, RoomSimulator
import srt
import re
from IPython.display import clear_output

#our libraries
from global_variables.training_vars import *
from global_variables.folders import *
from parashat_hashavua_dataset import *
from nikud_and_teamim import just_teamim,remove_nikud


In [6]:
processor = WhisperProcessor.from_pretrained("openai/whisper-" + BASE_MODEL_VERSION, language="hebrew", task="transcribe")

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [7]:
# # Check how many tokens needed for each nikud/taam
# chars = ['֑', '֒', '֓', '֔', '֕', '֖', '֗', '֘', '֙', '֚', '֛', '֜', '֝', '֞', '֟', '֠', '֡', '֢', '֣', '֤', '֥', '֦', '֧', '֨', '֩', '֪', '֫', '֬', '֭', '֮', '֯', 'ְ', 'ֱ', 'ֲ', 'ֳ', 'ִ', 'ֵ', 'ֶ', 'ַ', 'ָ', 'ֹ', 'ֺ', 'ֻ', 'ּ', 'ֽ', '־', 'ֿ', '׀', 'ׁ', 'ׂ', '׃', 'ׄ', 'ׅ', '׆', 'ׇ']
# for c in chars:
#     print(f"{c}: {len(processor.tokenizer.encode(c))-5}")


In [8]:
tokens_added = (len(processor.tokenizer.encode('֟'))==6) # check if the tokens were already added
if ADDTOKENS and not tokens_added: # add the tokens if they weren't already added
    
    teamim = ['֑', '֒', '֓', '֔', '֕', '֖', '֗', '֘', '֙', '֚', '֛', '֜', '֝', '֞', '֟', '֠', '֡', '֢', '֣', '֤', '֥', '֦', '֧', '֨', '֩', '֪', '֫', '֬', '֭', '֮', 'ֽ']
    if JUST_TEAMIM:
        new_tokens = [BASE_CHAR + c for c in teamim] # add the base char to the teamim (e.g. א֑)
    elif NIKUD:
        new_tokens = ['֑', '֒', '֓', '֔', '֕', '֖', '֗', '֘', '֙', '֚', '֛', '֜', '֝', '֞', '֟', '֠', '֡', '֢', '֣', '֤', '֥', '֦', '֧', '֨', '֩', '֪', '֫', '֬', '֭', '֮', '֯', 'ְ', 'ֱ', 'ֲ', 'ֳ', 'ִ', 'ֵ', 'ֶ', 'ַ', 'ָ', 'ֹ', 'ֺ', 'ֻ', 'ּ', 'ֽ', '־', 'ֿ', '׀', 'ׁ', 'ׂ', '׃', 'ׄ', 'ׅ', '׆', 'ׇ']
    else:
        new_tokens = teamim
    
    processor.tokenizer.add_tokens(new_tokens)

In [9]:
if COMBINE_DATA:
    train_data1 = parashat_hashavua_dataset(new_data = True, few_data=FASTTEST, train =True ,validation=False, random=RANDOM, num_of_words_in_sample=4, nusachim=NUSACHIM, augment=AUGMENT, processor=processor)
    train_data2 = parashat_hashavua_dataset(new_data = False, train =True ,validation=False, random=RANDOM, num_of_words_in_sample=13, augment=AUGMENT, prob_for_num_of_parts = [0.05, 0.05, 0.05, 0.05, 0.1, 0.15, 0.15, 0.2, 0.2], processor=processor)
    train_data = ConcatDataset([train_data1, train_data2])
else:
    if NEWDATA == "other":
        train_data = parashat_hashavua_dataset(new_data = NEWDATA, few_data=FASTTEST, train =True ,validation=False, random=RANDOM, num_of_words_in_sample=1, nusachim=NUSACHIM, augment=AUGMENT, processor=processor)
    elif NEWDATA:
        train_data = parashat_hashavua_dataset(new_data = True, few_data=FASTTEST, train =True ,validation=False, random=RANDOM, num_of_words_in_sample=4, nusachim=NUSACHIM, augment=AUGMENT, processor=processor)

    else:
        train_data = parashat_hashavua_dataset(new_data = False, train =True ,validation=False, random=RANDOM, num_of_words_in_sample=13, augment=AUGMENT, prob_for_num_of_parts = [0.05, 0.05, 0.05, 0.05, 0.1, 0.15, 0.15, 0.2, 0.2], processor=processor)

Loading audio nusach (1/1): 100%|██████████| 11/11 [00:00<00:00, 25.15it/s]

Num of missing files in audio nusach:  0


In [10]:
train_data[0]
processor.tokenizer.decode(train_data[0]['labels'])

'<|startoftranscript|><|he|><|transcribe|><|notimestamps|>בראש֖ית בר֣א אלה֑ים א֥ת השמ֖ים וא֥ת האֽרץ׃ והא֗רץ הית֥ה ת֙הו֙ וב֔הו וח֖שך על־ פנ֣י תה֑ום ור֣וח אלה֔ים מרח֖פת על־ פנ֥י המֽים׃ וי֥אמר אלה֖ים יה֣י א֑ור וֽיהי־ אֽור׃ וי֧רא אלה֛ים<|endoftext|>'

In [11]:
# check the data of Pocket Torah
# train_data.check_failed_files_of_old_data()

In [12]:
# play random from the train data
# train_data.print_and_play_word_by_index(random.randint(0, len(train_data.data)))


#validation of the data
# train_data.check_the_data() 
# remove the sample index:
# train_data.remove_word_by_index(32487)


In [13]:
if COMBINE_DATA:
    val_data1 = parashat_hashavua_dataset(new_data = True, few_data=FASTTEST, train=False ,validation=True,  num_of_words_in_sample=4, nusachim=NUSACHIM, processor=processor)
    val_data2 = parashat_hashavua_dataset(new_data = False, train=False ,validation=True, num_of_words_in_sample=13, random=RANDOM, processor=processor)
    val_data = ConcatDataset([val_data1, val_data2])
    
else:
    if NEWDATA == "other":
        val_data = parashat_hashavua_dataset(new_data = NEWDATA, few_data=FASTTEST, train=False ,validation=True,  num_of_words_in_sample=1, nusachim=NUSACHIM, processor=processor)
    elif NEWDATA:
        val_data = parashat_hashavua_dataset(new_data = True, few_data=FASTTEST, train=False ,validation=True,  num_of_words_in_sample=4, nusachim=NUSACHIM, processor=processor)
    else:
        val_data = parashat_hashavua_dataset(new_data = False, train=False ,validation=True, num_of_words_in_sample=13, random=RANDOM, processor=processor)
    

print(len(train_data))
print(len(val_data))

Loading audio nusach (1/1): 100%|██████████| 2/2 [00:00<00:00, 695.05it/s]

Num of missing files in audio nusach:  0
11
2


In [14]:
from dataclasses import dataclass
from typing import Any, Dict, List, Union

@dataclass
class DataCollatorSpeechSeq2SeqWithPadding:
    processor: Any

    def __call__(self, features: List[Dict[str, Union[List[int], torch.Tensor]]]) -> Dict[str, torch.Tensor]:
        # split inputs and labels since they have to be of different lengths and need different padding methods
        # first treat the audio inputs by simply returning torch tensors
        input_features = [{"input_features": feature["input_features"]} for feature in features]
        batch = self.processor.feature_extractor.pad(input_features, return_tensors="pt")

        # get the tokenized label sequences
        label_features = [{"input_ids": feature["labels"]} for feature in features]
        # pad the labels to max length
        labels_batch = self.processor.tokenizer.pad(label_features, return_tensors="pt")

        # replace padding with -100 to ignore loss correctly
        labels = labels_batch["input_ids"].masked_fill(labels_batch.attention_mask.ne(1), -100)

        # if bos token is appended in previous tokenization step,
        # cut bos token here as it's append later anyways
        if (labels[:, 0] == self.processor.tokenizer.bos_token_id).all().cpu().item():
            labels = labels[:, 1:]

        batch["labels"] = labels

        return batch

In [15]:
data_collator = DataCollatorSpeechSeq2SeqWithPadding(processor=processor)

In [16]:
import concurrent.futures
import evaluate
import time
import cantilLocations_evaluation


# # possible metrics : "wer", "cer", "bleu", "rouge", "sacrebleu", "sari":
# # 1. `wer`: Word Error Rate.
# # 2. `cer`: Character Error Rate.
# # 3. `bleu`: Bilingual Evaluation Understudy.
# # 4. `rouge`: Recall-Oriented Understudy for Gisting Evaluation.
# # 5. `sacrebleu`: A standardized BLEU score implementation for more consistent machine translation evaluation.
# # 6. `sari`: System Agnostic Refinement Index. 

WER_CALCULATOR = evaluate.load("wer")
def compute_metrics(pred):
    eval_list = cantilLocations_evaluation.calculate_precision_recall_f1_for_string_list_with_method_list
    pred_ids = pred.predictions
    label_ids = pred.label_ids
    
    # method to calculate the metrics(method can be "Exact", "Letter_Shift", "Word_Level", "Word_Shift")
    methods = ["Exact", "Letter_Shift", "Word_Level", "Word_Shift"]

    # replace -100 with the pad_token_id
    label_ids[label_ids == -100] = processor.tokenizer.pad_token_id
    
    start_time = time.time()
    
    # we do not want to group tokens when computing the metrics
    pred_str = processor.tokenizer.batch_decode(pred_ids, skip_special_tokens=True)
    label_str = processor.tokenizer.batch_decode(label_ids, skip_special_tokens=True)
    
    decode_time = time.time() - start_time
    
    # evaluate the metrics
    results = eval_list(pred_str, label_str, methods)
    
    
    
    # compute the average of each metric
    avg = {}
    for i in range(4):
        avg["avg_precision_" + methods[i]] = np.mean(results[i][0])
        avg["avg_recall_" + methods[i]] = np.mean(results[i][1])
        avg["avg_f1_" + methods[i]] = np.mean(results[i][2])
    
    precision_list_exact = results[methods.index("Exact")][0]
    recall_list_exact = results[methods.index("Exact")][1]
    f1_list_exact = results[methods.index("Exact")][2]
    
    # compute the median
    precision_median_exact = np.median(precision_list_exact)
    recall_median_exact = np.median(recall_list_exact)
    f1_median_exact = np.median(f1_list_exact)
    
    
    # max and min:
    precision_max_exact = np.max(precision_list_exact)
    recall_max_exact = np.max(recall_list_exact)
    f1_max_exact = np.max(f1_list_exact)
    best_index = np.argmax(f1_list_exact)
    
    f1_min = [0, 0, 0, 0]
    recall_min = [0, 0, 0, 0]
    precision_min = [0, 0, 0, 0]
    
    for i in range(4):
        precision_min[i] = np.min(results[i][0])
        recall_min[i] = np.min(results[i][1])
        f1_min[i] = np.min(results[i][2])
    
    worst_index = [np.argmin(results[i][2]) for i in range(4)] 
    
    
    
    start_time = time.time()
    # WER
    wer = 100 * WER_CALCULATOR.compute(predictions=pred_str, references=label_str)
    
    wer_time = time.time() - start_time
    
    best_pred = pred_str[best_index]
    best_label = label_str[best_index]
    worst_pred = [pred_str[worst_index[i]] for i in range(4)]
    worst_label = [label_str[worst_index[i]] for i in range(4)]
    
    # print
    # best:
    print(f"best f1 for {methods[0]}: {f1_max_exact}\nbest pred: {best_pred}\nbest label: {best_label}\n")
    
    # worst (the worst for each method):
    for i in range(4):
        print(f"worst f1 for {methods[i]}: {f1_min[i]}\nworst pred: {worst_pred[i]}\nworst label: {worst_label[i]}\n")
    
    
    
    print("Time taken for each part:")
    print(f"Decode calculation: {decode_time} seconds")
    print(f"WER calculation: {wer_time} seconds")
    
    # matric_dict = {"wer": wer, "precision": precision_avg, "recall": recall_avg, "f1": f1_avg, "precision_median": precision_median, "recall_median": recall_median, "f1_median": f1_median, "precision_max": precision_max, "recall_max": recall_max, "f1_max": f1_max, "precision_min": precision_min, "recall_min": recall_min, "f1_min": f1_min}
    
    # create the matric_dict with the metrics
    matric_dict = {"wer": wer}
    for i in range(4):
        matric_dict["avg_precision_" + methods[i]] = avg["avg_precision_" + methods[i]]
        matric_dict["avg_recall_" + methods[i]] = avg["avg_recall_" + methods[i]]
        matric_dict["avg_f1_" + methods[i]] = avg["avg_f1_" + methods[i]]
    matric_dict["precision_median_exact"] = precision_median_exact
    matric_dict["recall_median_exact"] = recall_median_exact
    matric_dict["f1_median_exact"] = f1_median_exact
    matric_dict["precision_max_exact"] = precision_max_exact
    matric_dict["recall_max_exact"] = recall_max_exact
    matric_dict["f1_max_exact"] = f1_max_exact
    for i in range(4):
        matric_dict["precision_min_" + methods[i]] = precision_min[i]
        matric_dict["recall_min_" + methods[i]] = recall_min[i]
        matric_dict["f1_min_" + methods[i]] = f1_min[i]
    # print(matric_dict)
    return matric_dict

In [17]:
from transformers import WhisperForConditionalGeneration

model = WhisperForConditionalGeneration.from_pretrained(BASE_MODEL_NAME, use_cache=False) # we can add "force_download=True" to download the model again

model.generation_config.language = "he"




# # initialize the last layer of the model:
# model.proj_out.__init__(model.proj_out.in_features, len(processor.tokenizer))

# # add dropout
if DROPOUT:
    model.config.attention_dropout = DROPOUT
    model.config.dropout = DROPOUT
    model.config.activation_dropout = DROPOUT


if ADDTOKENS:
    model.resize_token_embeddings(len(processor.tokenizer))

model.config.forced_decoder_ids = None
model.config.suppress_tokens = []

model.config.decoder_input_ids = None



Loading checkpoint shards: 100%|██████████| 2/2 [00:00<00:00,  2.24it/s]


In [18]:
# MODEL_NAME

In [19]:
from transformers import Seq2SeqTrainingArguments

training_args = Seq2SeqTrainingArguments(
    output_dir= MODEL_NAME,  # change to a repo name of your choice
    per_device_train_batch_size=BATCH_SIZE,
    gradient_accumulation_steps=1,  # increase by 2x for every 2x decrease in batch size
    learning_rate=LR, # was 1e-5
    warmup_steps=WARMUP_STEPS, # was 500
    max_steps=MAX_STEPS, # was 4000
    gradient_checkpointing=True,
    gradient_checkpointing_kwargs={'use_reentrant':False}, # I added that because UserWarning: "The default value of use_reentrant will be updated to be False in the future."
    fp16=torch.cuda.is_available(), # I added that because fp16 can't be use on CPU but on cuda
    evaluation_strategy="steps",
    per_device_eval_batch_size=32,
    predict_with_generate=True,
    generation_max_length=225,
    save_steps=SAVE_STEPS, 
    eval_steps=EVAL_STEPS,   
    logging_steps=25, 
    report_to=["tensorboard"],
    load_best_model_at_end=True,
    metric_for_best_model= "avg_f1_Exact",# "avg_f1_..." like "avg_f1_Exact"
    greater_is_better=True, # if we use f1 score in eval so greater is better
    push_to_hub=True,
    # I added the dataloader_prefetch_factor to support newer versions of torch (now it must be int and not None. and the default is 2).
    dataloader_prefetch_factor=2, # support newer versions of torch
    dataloader_num_workers=1, # parallelize the data loading
    weight_decay=WEIGHT_DECAY,
    run_name=RUN_NAME, # It doesn't work
)


/opt/conda/lib/python3.10/site-packages/transformers/training_args.py:1474: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [ ]:
from transformers import Seq2SeqTrainer, TrainerCallback


class EvaluateFirstStepCallback(TrainerCallback):
    def on_step_begin(self, args, state, control, model, tokenizer, optimizer, lr_scheduler, train_dataloader, eval_dataloader, **kwargs):
        if state.global_step == 0:
            control.should_evaluate = True



trainer = Seq2SeqTrainer(
    args=training_args,
    model=model,
    train_dataset=train_data,
    eval_dataset=val_data,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    tokenizer=processor.feature_extractor,
    callbacks = [EvaluateFirstStepCallback()] if EVALUATE_FIRST_STEP else None
)



In [ ]:
processor.save_pretrained(training_args.output_dir)

In [ ]:
def flags_warnings():
    if FASTTEST:
        for i in range(10):
            print("!!!TEST-MODE!!! \t \t to test the code only")

    if not ADDTOKENS:
        print("!!!ADDTOKENS==False!!!")

    if not NEWDATA:
        print("!!!NEWDATA==False!!!")

In [ ]:
flags_warnings()

trainer_state = trainer.train()

In [ ]:
kwargs = {
    # "dataset_args": "config: he, split: test",
    "language": "he",
    "model_name": "he-cantillation",
    "finetuned_from": BASE_MODEL_NAME,
    "tasks": "automatic-speech-recognition-cantillation",
    "tags": "hf-asr-leaderboard",
}

In [ ]:
trainer.save_model()

In [ ]:
trainer.push_to_hub(**kwargs)
# processor.push_to_hub("cantillation" +training_args.output_dir[1:])

In [ ]:
trainer.lr_scheduler.get_lr()

In [ ]:
processor.tokenizer.decode(train_data[26]["labels"])

In [ ]:
# !pip install peft

In [24]:
# train LoRA model


training_args = Seq2SeqTrainingArguments(
    output_dir= MODEL_NAME,  # change to a repo name of your choice
    per_device_train_batch_size=BATCH_SIZE,
    gradient_accumulation_steps=1,  # increase by 2x for every 2x decrease in batch size
    learning_rate=1e-3, # was 1e-5
    warmup_steps=WARMUP_STEPS, 
    max_steps=MAX_STEPS,
    # num_train_epochs=2,
    gradient_checkpointing=True,
    gradient_checkpointing_kwargs={'use_reentrant':False}, # I added that because UserWarning: "The default value of use_reentrant will be updated to be False in the future."
    fp16=torch.cuda.is_available(), # I added that because fp16 can't be use on CPU but on cuda
    evaluation_strategy="steps",
    per_device_eval_batch_size=32,
    predict_with_generate=True,
    generation_max_length=225,
    save_steps=SAVE_STEPS, 
    eval_steps=2,   
    logging_steps=2, 
    report_to=["tensorboard"],
    load_best_model_at_end=True,
    metric_for_best_model= "avg_f1_Exact",# "avg_f1_..." like "avg_f1_Exact"
    greater_is_better=True, # if we use f1 score in eval so greater is better
    push_to_hub=True,
    # I added the dataloader_prefetch_factor to support newer versions of torch (now it must be int and not None. and the default is 2).
    dataloader_prefetch_factor=2, # support newer versions of torch
    dataloader_num_workers=1, # parallelize the data loading
    weight_decay=WEIGHT_DECAY,
    remove_unused_columns=False,  # required as the PeftModel forward doesn't have the signature of the wrapped model's forward
    
)

from peft import LoraConfig, PeftModel, LoraModel, LoraConfig, get_peft_model

lora_config = LoraConfig(r=16, lora_alpha=32, target_modules=["q_proj", "v_proj"], lora_dropout=0.05, bias="none")

model = get_peft_model(model, lora_config)
model.print_trainable_parameters()

from transformers import Seq2SeqTrainer, TrainerCallback, TrainingArguments, TrainerState, TrainerControl
from transformers.trainer_utils import PREFIX_CHECKPOINT_DIR


# This callback helps to save only the adapter weights and remove the base model weights.
class SavePeftModelCallback(TrainerCallback):
    def on_save(
        self,
        args: TrainingArguments,
        state: TrainerState,
        control: TrainerControl,
        **kwargs,
    ):
        checkpoint_folder = os.path.join(args.output_dir, f"{PREFIX_CHECKPOINT_DIR}-{state.global_step}")

        peft_model_path = os.path.join(checkpoint_folder, "adapter_model")
        kwargs["model"].save_pretrained(peft_model_path)

        pytorch_model_path = os.path.join(checkpoint_folder, "pytorch_model.bin")
        if os.path.exists(pytorch_model_path):
            os.remove(pytorch_model_path)
        return control

from transformers import Seq2SeqTrainer, TrainerCallback

class EvaluateFirstStepCallback(TrainerCallback):
    def on_step_begin(self, args, state, control, model, tokenizer, optimizer, lr_scheduler, train_dataloader, eval_dataloader, **kwargs):
        if state.global_step == 0:
            control.should_evaluate = True



trainer = Seq2SeqTrainer(
    args=training_args,
    model=model,
    train_dataset=train_data,
    eval_dataset=val_data,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    tokenizer=processor.feature_extractor,
    callbacks = [SavePeftModelCallback]
)



model.config.use_cache = False 
trainer.train()
trainer.save_model()


/opt/conda/lib/python3.10/site-packages/transformers/training_args.py:1474: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
max_steps is given, it will override any value given in num_train_epochs


trainable params: 7,864,320 || all params: 1,551,208,960 || trainable%: 0.5070


Step,Training Loss,Validation Loss,Wer,Avg Precision Exact,Avg Recall Exact,Avg F1 Exact,Avg Precision Letter Shift,Avg Recall Letter Shift,Avg F1 Letter Shift,Avg Precision Word Level,Avg Recall Word Level,Avg F1 Word Level,Avg Precision Word Shift,Avg Recall Word Shift,Avg F1 Word Shift,Precision Median Exact,Recall Median Exact,F1 Median Exact,Precision Max Exact,Recall Max Exact,F1 Max Exact,Precision Min Exact,Recall Min Exact,F1 Min Exact,Precision Min Letter Shift,Recall Min Letter Shift,F1 Min Letter Shift,Precision Min Word Level,Recall Min Word Level,F1 Min Word Level,Precision Min Word Shift,Recall Min Word Shift,F1 Min Word Shift
2,13.842200,1.398150,78.947368,0.250000,0.250000,0.250000,0.272727,0.272727,0.272727,0.285714,0.272727,0.279070,0.362121,0.372378,0.366883,0.250000,0.250000,0.250000,0.500000,0.500000,0.500000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.133333,0.153846,0.142857
4,10.770800,1.427956,78.947368,0.250000,0.261905,0.255814,0.272727,0.285714,0.279070,0.285714,0.285714,0.285714,0.362121,0.386447,0.373754,0.250000,0.261905,0.255814,0.500000,0.523810,0.511628,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.133333,0.153846,0.142857
6,6.556900,1.251958,73.684211,0.227273,0.238095,0.232558,0.250000,0.261905,0.255814,0.261905,0.261905,0.261905,0.372727,0.401099,0.386213,0.227273,0.238095,0.232558,0.454545,0.476190,0.465116,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.200000,0.230769,0.214286
8,6.016700,3.818477,97.368421,0.000000,0.000000,0.000000,0.033333,0.166667,0.055556,0.090476,0.500000,0.152778,0.094156,0.500000,0.157647,0.000000,0.000000,0.000000,0,0,0,0,0,0,0.000000,0.000000,0.000000,0.047619,0.333333,0.083333,0.045455,0.333333,0.080000


best f1 for Exact: 0.5
best pred: שֽלצ֨ו המ֜ים למֽינהם֙ וא֨ת כל־ע֤וף כנ֣ף למינ֔הו וי֥ג אלה֖ים כי־טֽוב׃ ויב֣רך את֔ם אלה֣ים לאמ֑ר פר֣ו וב֗ו ומלא֨ו את־המ֧ים בימ֖ים והעֽוף׃
best label: שרצ֨ו המ֜ים למֽינה֗ם וא֨ת כל־ ע֤וף כנף֙ למינ֔הו וי֥רא אלה֖ים כי־ טֽוב׃ ויב֧רך את֛ם אלה֖ים לאמ֑ר פר֣ו ורב֗ו ומלא֤ו את־ המ֙ים֙ בימ֔ים והע֖וף 

worst f1 for Exact: 0.0
worst pred: י֣רב בא֑רץ וי֣רב בב֡קר י֣ום חמישֽי׃ וי֣אמר אלהים֩ תוצ֜א אל־ע֜ץ נ֖פש חי֣ה למינ֑ה
worst label: י֥רב באֽרץ׃ וֽיהי־ ע֥רב וֽיהי־ ב֖קר י֥ום חמישֽי׃ וי֣אמר אלה֗ים תוצ֨א הא֜רץ נ֤פש חיה֙ למינ֔ה 

worst f1 for Letter_Shift: 0.0
worst pred: י֣רב בא֑רץ וי֣רב בב֡קר י֣ום חמישֽי׃ וי֣אמר אלהים֩ תוצ֜א אל־ע֜ץ נ֖פש חי֣ה למינ֑ה
worst label: י֥רב באֽרץ׃ וֽיהי־ ע֥רב וֽיהי־ ב֖קר י֥ום חמישֽי׃ וי֣אמר אלה֗ים תוצ֨א הא֜רץ נ֤פש חיה֙ למינ֔ה 

worst f1 for Word_Level: 0.0
worst pred: י֣רב בא֑רץ וי֣רב בב֡קר י֣ום חמישֽי׃ וי֣אמר אלהים֩ תוצ֜א אל־ע֜ץ נ֖פש חי֣ה למינ֑ה
worst label: י֥רב באֽרץ׃ וֽיהי־ ע֥רב וֽיהי־ ב֖קר י֥ום חמישֽי׃ וי֣אמר אלה֗ים תוצ֨א הא֜רץ נ֤פש חיה֙ למינ֔ה

KeyboardInterrupt: 

In [ ]:

from datetime import datetime


def log_training_to_markdown_file(training_args, training_loss, epoch, step, validation_loss, f1, recall, precision, filename="training_log.md"):
    # Get the current date and time
    now = datetime.now()

    # Format the date and time as a string
    date_time = now.strftime("%m/%d/%Y, %H:%M:%S")

    with open(filename, 'a') as f:
        f.write(f"| {date_time} | {training_args.output_dir } | {training_args.per_device_train_batch_size} | {training_args.gradient_accumulation_steps} | {training_args.learning_rate} | {training_args.warmup_steps} | {training_args.max_steps} | {training_args.gradient_checkpointing} | {training_args.gradient_checkpointing_kwargs} | {training_args.fp16} | {training_args.evaluation_strategy} | {training_args.per_device_eval_batch_size} | {training_args.predict_with_generate} | {training_args.generation_max_length} | {training_args.save_steps} | {training_args.eval_steps} | {training_args.logging_steps} | {training_args.report_to} | {training_args.load_best_model_at_end} | {training_args.metric_for_best_model} | {training_args.greater_is_better} | {training_args.push_to_hub} | {training_loss} | {epoch} | {step} | {validation_loss} | {f1} | {recall} | {precision} |\n")

def create_markdown_file_with_headers(filename="./markdown_files/training_log_new.md"):
    with open(filename, 'w') as f:
        f.write("| Date Time | Repo Name | Batch Size | Gradient Accumulation Steps | Learning Rate | Warmup Steps | Max Steps | Gradient Checkpointing | Gradient Checkpointing Kwargs | FP16 | Evaluation Strategy | Eval Batch Size | Predict with Generate | Max Length | Save Steps | Eval Steps | Logging Steps | Report To | Load Best Model at End | Metric for Best Model | Greater is Better | Push to Hub | Training Loss | Epoch | Step | Validation Loss | f1 | recall | precision |\n")
        f.write("|---|---|---|---|---|---|---|---|---|---|---|---|---|---|---|---|---|---|---|---|---|---|---|---|---|---|----|---|---|\n")

# Create the Markdown file with headers
#create_markdown_file_with_headers()
        

def get_logs_with_step(trainer, step = 1500):
    # Initialize an empty dictionary to store the merged logs
    merged_logs_with_step = {}

    # Iterate over the log history
    for log in trainer.state.log_history:
        # Check if the 'step' key exists in the log and if it equals the provided step
        if 'step' in log and log['step'] == step:
            # If it does, merge the log into the merged_logs_with_step dictionary
            merged_logs_with_step.update(log)

    # Return the merged logs
    return merged_logs_with_step


# Get the training loss
training_loss = trainer_state.training_loss
# Get the step and epoch from the TrainerState
step = trainer.state.global_step
epoch = trainer.state.epoch

# Get the log history at the specified step
history = get_logs_with_step(trainer,training_args.max_steps)
# Get the evaluation details from the log history
validation_loss = history['eval_loss']
f1 = history['eval_avg_f1_Exact']
recall = history['eval_avg_recall_Exact']
precision = history['eval_avg_precision_Exact']

# Log the training details
log_training_to_markdown_file(training_args, training_loss, epoch, step, validation_loss, f1, recall, precision, filename="./markdown_files/training_log_new.md")


In [ ]:
#load the markdown file
from IPython.display import Markdown

# Open the file in read mode
with open('./markdown_files/training_log_new.md', 'r') as file:
    # Read the content of the file
    content = file.read()

# Display the content as Markdown
display(Markdown(content))

| Model Name | Model Name | data | steps | lr |
|----------|----------|----------|--------|--------|
| whisper-medium-he-teamim-base | medium | all | 10,000 | 3e-5 |
| whisper-medium-he-teamim-ashkenazi-01 | base | ashkenazi | 9,000 | 1e-6 |




In [ ]:
# run a web server to see the tensorboard
!tensorboard --logdir ./whisper-medium-he-teamim-aviv-base --port 6006 --bind_all

# Test the model

In [ ]:
# load the model
model = WhisperForConditionalGeneration.from_pretrained(MODEL_NAME)
processor = WhisperProcessor.from_pretrained(MODEL_NAME)

# load the test data
test_data = parashat_hashavua_dataset(few_data=FASTTEST, train=False ,validation=False, test=True,  num_of_words_in_sample=4, nusachim=NUSACHIM, processor=processor)

# create the data collator
data_collator = DataCollatorSpeechSeq2SeqWithPadding(processor=processor)


In [ ]:
from transformers import Seq2SeqTrainer
trainer = Seq2SeqTrainer(
    args=training_args,
    model=model,
    eval_dataset=test_data,
    data_collator=data_collator,
    compute_metrics=compute_metrics,
    tokenizer=processor.feature_extractor,
)

# evaluate the model
results = trainer.evaluate() # we use evaluate to get the metrics
print(results)
# save the results to a json file
# create the results file
with open(f"results_{MODEL_NAME.split('/')[-1]}.json", 'w') as f:
    json.dump(results, f, indent=4)


In [ ]:
{'eval_loss': 0.7434155344963074, 'eval_wer': 12.154756685664182, 'eval_avg_precision_Exact': 0.9045080330977024, 'eval_avg_recall_Exact': 0.9046995984902677, 'eval_avg_f1_Exact': 0.9041808511811215, 'eval_avg_precision_Letter_Shift': 0.9262739837538204, 'eval_avg_recall_Letter_Shift': 0.9265417247288052, 'eval_avg_f1_Letter_Shift': 0.9259759561670436, 'eval_avg_precision_Word_Level': 0.9287635843313802, 'eval_avg_recall_Word_Level': 0.9291329677264785, 'eval_avg_f1_Word_Level': 0.928526987815817, 'eval_avg_precision_Word_Shift': 0.9744189603225837, 'eval_avg_recall_Word_Shift': 0.9756642194342305, 'eval_avg_f1_Word_Shift': 0.9746023104771994, 'eval_precision_median_exact': 1.0, 'eval_recall_median_exact': 1.0, 'eval_f1_median_exact': 1.0, 'eval_precision_max_exact': 1.0, 'eval_recall_max_exact': 1.0, 'eval_f1_max_exact': 1.0, 'eval_precision_min_Exact': 0.0, 'eval_recall_min_Exact': 0.0, 'eval_f1_min_Exact': 0.0, 'eval_precision_min_Letter_Shift': 0.0, 'eval_recall_min_Letter_Shift': 0.0, 'eval_f1_min_Letter_Shift': 0.0, 'eval_precision_min_Word_Level': 0.0, 'eval_recall_min_Word_Level': 0.0, 'eval_f1_min_Word_Level': 0.0, 'eval_precision_min_Word_Shift': 0.0, 'eval_recall_min_Word_Shift': 0.0, 'eval_f1_min_Word_Shift': 0.0, 'eval_runtime': 1534.2005, 'eval_samples_per_second': 1.755, 'eval_steps_per_second': 0.055}